In [1]:
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar
from matplotlib.colors import Normalize, TwoSlopeNorm
import cmocean
import cartopy.crs as ccrs
import cartopy
import seaborn as sns
from matplotlib import rcParams, cycler
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from xeofs.xarray import EOF

import scipy
from scipy import signal
from shapely.geometry import mapping
from xarrayutils.utils import linear_trend, xr_linregress
import pandas as pd
import geopandas as gpd

/var/folders/6b/9ftjhcg16g19w33zvk7zd7mm0131yc/T/ipykernel_74855/3092812729.py:22: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [ ]:

main_dir = Path.cwd().parent # Main directory path of project repository - all filepaths are relative to this

# File path directories
DIR_external = 'data/external/'

# DATASET FILEPATHS
# Basal melt observations from Paolo 2023
DIR_basalMeltObs = 'data/external/Paolo2023/'
# Ocean model output - E3SM (SORRMv2.1.ISMF), data received from Darin Comeau / Matt Hoffman at LANL
DIR_SORRMv21 = 'data/external/SORRMv2.1.ISMF/regridded_output/'

# DATA FILENAMES
FILE_MeltDraftObs = 'ANT_G1920V01_IceShelfMeltDraft.nc'
FILE_SORRMv21 = 'Regridded_SORRMv2.1.ISMF.FULL.nc'
FILE_SORRMv21_DETRENDED = 'SORRMv21_detrended.nc'
FILE_iceShelvesShape = 'iceShelves.geojson'

# INTERIM GENERATED FILEPATHS
DIR_basalMeltObs_Interim = 'data/interim/Paolo2023/iceShelves_dedraft/iceShelfRegions/'
DIR_SORRMv21_Interim = 'data/interim/SORRMv2.1.ISMF/iceShelves_dedraft/iceShelfRegions/'


In [ ]:
norm_eofs = xr.open_dataset(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_norm_eofs.nc" )
norm_pcs = xr.open_dataset(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_norm_pcs.nc" )
norm_varexpl = xr.open_dataset(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_norm_varexpl.nc" )

norm_eofs = norm_eofs.EOFs
norm_pcs = norm_pcs.PCs
nmodes = norm_eofs.mode.shape[0]

In [ ]:
plt.figure(figsize=(75,8))
#norm_pcs.PCs[:,0].plot()
norm_pcs.PCs[:,1].plot()
norm_pcs.PCs[:,5].plot()

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 2
t_length = norm_pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,norm_pcs.shape[0],norm_pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = norm_pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_fourier = np.fft.rfft(fl)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
    print('calculated ifft for realization {}, all modes'.format(i))

In [ ]:
t_start = 1500
t_end = 3000

# norm_pcs_trend = 


plt.figure(figsize=(35,8))
norm_pcs[t_start:t_end,1].plot(linewidth=2)
norm_pcs[t_start:t_end,4].plot(linewidth=2)
norm_pcs[t_start:t_end,7].plot(linewidth=2)
norm_pcs[t_start:t_end,10].plot(linewidth=2)

In [ ]:
from statsmodels.tsa.seasonal import STL
# fig, ax = plt.subplots(111, figsize=(45,8))
mode_list = np.array([1,4,7,10])

plt.figure(figsize=(45,8))
for i in mode_list:
    res = STL(norm_pcs[t_start:t_end,i],period=12).fit()
    plt.plot(res.trend,linewidth=2.5)

In [ ]:
plt.figure(figsize=(25,8))
plt.plot(norm_pcs[t_start:t_end,10])
plt.plot(res.trend)

In [ ]:
# Diagnostic plot of generated PCs

t_start = 0
t_end = 6000

nmodes_plot = 10 # Number of modes to plot
nrealizations_to_plot = n_realizations # to be lesser than absolute total number, defined in the Fourier randomization step

sns.set_theme(style="white")
fig=plt.figure(figsize=(25,5*nmodes_plot))

gs = GridSpec(nmodes_plot, 2, width_ratios=[4, 2])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(nmodes_plot)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(nmodes_plot)]

for i, (a0,a1) in enumerate(zip(ax0,ax1)):
    for n_realization in range(0,nrealizations_to_plot):
        a0.plot(new_fl[n_realization,t_start:t_end,i],color='b', linewidth=0.5)
        a1.psd(new_fl[n_realization,t_start:t_end,i],color='b', linewidth=0.5)
    a0.plot(new_fl[0,t_start:t_end,i],color='b', linewidth=0.25,label='Randomized')
    a1.psd(new_fl[0,t_start:t_end,i],color='b', linewidth=0.25,label='Randomized')
    a0.plot(norm_pcs[t_start:t_end,i],color='k', linewidth=2.5,label='Original')
    a1.psd(norm_pcs[t_start:t_end,i],color='k', linewidth=2.5,label='Original')
    a0.set_title('PC for EOF mode {}'.format(i+1))
    a1.set_title('PSD for PC mode {}'.format(i+1))
    a1.set_xlabel('')

a0.set_xlabel('Time (months)')
a1.set_xlabel('Frequency')
plt.legend();

In [ ]:
#modified to not return f - in calculation of RMSE, only Px required
def psd_calc_grid(data,y,x):
    f, Px = scipy.signal.welch(data[:,y,x])
    return Px

def time_series(clipped_data):
    clipped_ts = clipped_data.sum(['y','x'])
    return clipped_ts

# Reconstruct flux dataset using phase randomized PCs
# This section is to be called iteratively for ensemble runs with multiple realizations
# This method also takes 'modes' as a parameter: 
# which is used to reconstruct dataset with different number of selected modes

def generate_data(n_realization,mode,mode_skip):
    # mode can be any int in (1,nmodes), for cases 
    # when dimensionality reduction is preferred on the reconstructed dataset
    flux_reconstr = norm_model.reconstruct_randomized_X(new_fl[n_realization],slice(1,mode,mode_skip))
    #flux_reconstr = flux_reconstr.dropna('time',how='all')
    #flux_reconstr = flux_reconstr.dropna('y',how='all')
    #flux_reconstr = flux_reconstr.dropna('x',how='all')
    #flux_reconstr = flux_reconstr.drop("month")
    return flux_reconstr

def clip_data(total_data, basin):
    clipped_data = total_data.rio.clip(icems.loc[[basin],'geometry'].apply(mapping))
    #clipped_data = clipped_data.dropna('time',how='all')
    #clipped_data = clipped_data.dropna('y',how='all')
    #clipped_data = clipped_data.dropna('x',how='all')
    clipped_data = clipped_data.drop("month")
    return clipped_data

In [ ]:
flux_clean_normalized = xr.open_dataset(main_dir / "data/interim/SORRMv2.1.ISMF/" / "flux_clean_6000_normalized.nc")
flux_clean_normalized = flux_clean_normalized.flux

norm_model = EOF(flux_clean_normalized)
norm_model.solve()

In [ ]:
norm_model

In [ ]:
import pickle
file_pi = open(str(main_dir / "data/interim/SORRMv2.1.ISMF/" / "norm_model.obj"), 'wb') 
pickle.dump(norm_model, file_pi)

In [ ]:
file_pi.close()

In [ ]:
file_pi2 = open(str(main_dir / "data/interim/SORRMv2.1.ISMF/" / "norm_model.obj"), 'rb')
norm_model2 = pickle.load(file_pi2)

In [ ]:
sorrmv21_clean = xr.open_dataset(main_dir / DIR_SORRMv21_Interim / "sorrmv21_clean.nc")
sorrmv21_clean = sorrmv21_clean.rename({"__xarray_dataarray_variable__":"flux", "Time":"time"})
flux_clean = sorrmv21_clean.flux[3000:9000]

flux_clean_tstd = xr.open_dataset(main_dir / "data/interim/SORRMv2.1.ISMF/" / "flux_clean_6000_tstd.nc")
flux_clean_tmean = flux_clean.mean('time')
flux_clean_tmean.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/" / "flux_clean_6000_tmean.nc")

In [ ]:
# Generate dataset realizations

## Standard EOF/PCA implementation
# Can use the xeofs-rand package, or directly generate using sklearn PCA.

for i in range(n_realizations):
    flux_reconstr = generate_data(i, 6000, 1)
    flux_reconstr = (flux_reconstr*flux_clean_tstd)+flux_clean_tmean
    # melt_reconstr = flux_reconstr*sec_per_year/rho_fw
    flux_reconstr = flux_reconstr.rename('flux_rec{}'.format(n_realizations))
    flux_reconstr.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/SORRM_6000_REC/flux_REC{}.nc".format(i))
    print('reconstructed realization # {}'.format(i))